In [19]:
import bisect
from collections import defaultdict
import csv
import pandas as pd
import json
import numpy as np

In [ ]:
all_wavelengths = [412.0, 440.0, 488.0, 510.0, 532.0, 555.0, 630.0, 650.0, 676.0] # 715.0 is included in a lot of the data but out of visible spectrum?
table = defaultdict(lambda: defaultdict(dict))

In [ ]:
def wavelength_weight(index : int) -> float:
    wavelength = all_wavelengths[index]
    weight = 0.0
    if index == 0: 
        weight += wavelength - 400
    else:
        weight += (wavelength - all_wavelengths[index - 1]) / 2
    if index == len(all_wavelengths) - 1:
        weight += 700 - wavelength
    else:
        weight += (all_wavelengths[index + 1] - wavelength) / 2
    return weight

all_wavelength_weights = [wavelength_weight(i) for i in range(len(all_wavelengths))]
all_wavelength_weights

In [22]:
with open('b.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in list(reader)[1:]:
        wavelength = float(row[1])
        if wavelength not in all_wavelengths:
            continue
        scattering = float(row[2])
        jerlov_water_type = row[3]
        table[jerlov_water_type][wavelength]['sigma_s'] = scattering

In [23]:
with open('a.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in list(reader)[1:]:
        wavelength = float(row[1])
        if wavelength not in all_wavelengths:
            continue
        absorbtion = float(row[2])
        jerlov_water_type = row[3]
        scattering = table[jerlov_water_type][wavelength]['sigma_s']
        extinction = scattering + absorbtion
        table[jerlov_water_type][wavelength]['sigma_t'] = float(extinction)

In [24]:
df_kd = pd.read_csv('kd.csv', index_col=0)

In [25]:
for jerlov_water_type, val in table.items():
        for wavelength, properties in val.items():
            kd = df_kd.loc[wavelength, jerlov_water_type]
            table[jerlov_water_type][wavelength]['kd'] = kd

In [ ]:
with open('ciesensitivity.csv', 'r'):
    sensitivities = [np.array([float(val) for val in line]) for line in csv.reader(open('ciesensitivity.csv'))]

cie_range = [380, 830]

def lerp(a, b, t):
    return a + t * (b - a)

def get_sensitivity(wavelength):
    index = (wavelength - cie_range[0]) / (cie_range[1] - cie_range[0]) * len(sensitivities)
    return lerp(sensitivities[int(index)], sensitivities[int(index) + 1], index % 1)

all_sensitivities = [get_sensitivity(wavelength) for wavelength in all_wavelengths]

In [27]:
def fill_missing(table):        
    for jerlov_water_type, val in table.items():
        water_type_wavelengths = sorted(val.keys())
        
        for wavelength in all_wavelengths:
            if wavelength not in water_type_wavelengths:
                # Lerp in between 2 closest wavelengths, or use the closest wavelength if out of bounds
                bisect_index = bisect.bisect(water_type_wavelengths, wavelength)
                if bisect_index == 0: 
                    val[wavelength] = val[water_type_wavelengths[0]]
                elif bisect_index == len(water_type_wavelengths):
                    val[wavelength] = val[water_type_wavelengths[-1]]
                else:
                    prev_wavelength = water_type_wavelengths[bisect_index - 1]
                    next_wavelength = water_type_wavelengths[bisect_index]
                    fract = (wavelength - prev_wavelength) / (next_wavelength - prev_wavelength)
                    prev_properties = val[prev_wavelength]
                    next_properties = val[next_wavelength]
                    properties = {
                        key: prev_properties[key] + fract * (next_properties[key] - prev_properties[key])
                        for key in prev_properties.keys()
                    }
                    val[wavelength] = properties
fill_missing(table)
table

defaultdict(<function __main__.<lambda>()>,
            {'b_IA': defaultdict(dict,
                         {412.0: {'sigma_s': 0.1175,
                           'sigma_t': 0.1298,
                           'kd': 0.0346},
                          440.0: {'sigma_s': 0.10569999999999999,
                           'sigma_t': 0.1291,
                           'kd': 0.028},
                          488.0: {'sigma_s': 0.1109,
                           'sigma_t': 0.1242,
                           'kd': 0.0286},
                          510.0: {'sigma_s': 0.09550000000000003,
                           'sigma_t': 0.13740000000000002,
                           'kd': 0.0384},
                          555.0: {'sigma_s': 0.08819999999999997,
                           'sigma_t': 0.1538,
                           'kd': 0.0724},
                          630.0: {'sigma_s': 0.07090000000000007,
                           'sigma_t': 0.3694,
                           'kd': 0.322},
        

In [28]:
jerlov_water_data = {
    jerlov_water_type: [val[wavelength] for wavelength in all_wavelengths]
    for jerlov_water_type, val in table.items()
}
jerlov_water_data

{'b_IA': [{'sigma_s': 0.1175, 'sigma_t': 0.1298, 'kd': 0.0346},
  {'sigma_s': 0.10569999999999999, 'sigma_t': 0.1291, 'kd': 0.028},
  {'sigma_s': 0.1109, 'sigma_t': 0.1242, 'kd': 0.0286},
  {'sigma_s': 0.09550000000000003,
   'sigma_t': 0.13740000000000002,
   'kd': 0.0384},
  {'sigma_s': 0.09193111111111112,
   'sigma_t': 0.14541777777777778,
   'kd': 0.05502222222222222},
  {'sigma_s': 0.08819999999999997, 'sigma_t': 0.1538, 'kd': 0.0724},
  {'sigma_s': 0.07090000000000007, 'sigma_t': 0.3694, 'kd': 0.322},
  {'sigma_s': 0.07450000000000001, 'sigma_t': 0.4194, 'kd': 0.37},
  {'sigma_s': 0.07400000000000012,
   'sigma_t': 0.5322000000000001,
   'kd': 0.4356}],
 'c_IB': [{'sigma_s': 0.14115, 'sigma_t': 0.18450833333333333, 'kd': 0.0467},
  {'sigma_s': 0.134925, 'sigma_t': 0.17723333333333333, 'kd': 0.0384},
  {'sigma_s': 0.13710833333333336,
   'sigma_t': 0.16525833333333337,
   'kd': 0.0377},
  {'sigma_s': 0.1266, 'sigma_t': 0.17593333333333333, 'kd': 0.0468},
  {'sigma_s': 0.086725, '

In [30]:
output = {
    'wavelengths': all_wavelengths,
    'wavelengthWeights': all_wavelength_weights,
    'jerlovWaterProps': jerlov_water_data,
    'sensitivities': {'cie' : [sensitivity.tolist() for sensitivity in all_sensitivities]}
}

json.dump(output, open('data.json', 'w'), indent=4)